### IMDB Sentiment Analysis
This project uses a Recurrent Neural Network with LSTM using a sequence-to-sequence model to predict sentiment classification.  
Dataset can be found [here.]("https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews?resource=download")

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np

### Preprocessing data

In [14]:
dataset = pd.read_csv("IMDB_Dataset.csv")

In [15]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [16]:
dataset['review'][0]
#to look at the general content and length of a review

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [17]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [18]:
dataset["sentiment"].value_counts()
#this dataset is balanced as it has an equalsplit for positive and negative sentiments

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

### One hot encoding

In [19]:
#one hot encoding for labels
data_encoded = pd.get_dummies(dataset, columns=["sentiment", ], drop_first=True, dtype=int)

In [20]:
data_encoded.head()
#positive values are reresented by 1, and negative values are represented by 0

,review,sentiment_positive
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [21]:
data_encoded = data_encoded.rename(columns={"sentiment_positive": "sentiment"})

In [22]:
data_encoded["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

### Splitting into training and testing data

In [23]:
train, test = train_test_split(data_encoded, test_size=0.2, random_state=24)
#only 2 columns in the data so no need to divide the dataset further into X and Y here

In [24]:
train.shape

(40000, 2)

In [25]:
test.shape

(10000, 2)

In [26]:
tokenizer = Tokenizer(num_words=5000) #IDs of the 5000 most frequent words are returned
tokenizer.fit_on_texts(train["review"])

In [27]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train["review"]), maxlen=300)
X_test = pad_sequences(tokenizer.texts_to_sequences(test["review"]), maxlen=300)

In [28]:
X_train
#array of tekenized text values

array([[   0,    0,    0, ...,  207,   26,  107],
       [   0,    0,    0, ...,  348,  788,  156],
       [   1,  167,   56, ...,    8,    3, 3320],
       ...,
       [   0,    0,    0, ...,    3,  226,  336],
       [   0,    0,    0, ...,  230,  265,    5],
       [   0,    0,    0, ..., 1564,  107,    9]])

In [29]:
Y_train = train["sentiment"]
Y_test = test["sentiment"]

In [30]:
Y_train

31724    1
9389     1
17228    1
41843    0
31584    0
        ..
21633    0
19857    1
14528    1
899      0
45474    1
Name: sentiment, Length: 40000, dtype: int32

### Build model 

In [31]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=300))
model.add(LSTM(128, dropout=0.20, recurrent_dropout=0.20))
model.add(Dense(1, activation = "sigmoid")) #because this is a binary classification, sigmoid is used

C:\Users\Kat\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [35]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (16, 300, 128)              │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (16, 128)                   │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (16, 1)                     │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,315,141 (8.83 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,543,428 (5.89 MB)

In [33]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [34]:
model.fit(X_train, Y_train, epochs=5, batch_size=16, validation_split=0.2)

Epoch 1/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 142s 70ms/step - accuracy: 0.7069 - loss: 0.5586 - val_accuracy: 0.8359 - val_loss: 0.3841
Epoch 2/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 141s 70ms/step - accuracy: 0.8589 - loss: 0.3353 - val_accuracy: 0.8851 - val_loss: 0.2795
Epoch 3/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 141s 71ms/step - accuracy: 0.9097 - loss: 0.2267 - val_accuracy: 0.8967 - val_loss: 0.2570
Epoch 4/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 140s 70ms/step - accuracy: 0.9398 - loss: 0.1602 - val_accuracy: 0.8945 - val_loss: 0.2685
Epoch 5/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 141s 70ms/step - accuracy: 0.9566 - loss: 0.1204 - val_accuracy: 0.8946 - val_loss: 0.2830


In [36]:
model.save("sentiment_model.keras")

In [37]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

### Evaluate the model

In [38]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.8896 - loss: 0.2936


In [39]:
print(f'Accuracy: {accuracy}, Loss: {loss}')

Accuracy: 0.8913999795913696, Loss: 0.28533563017845154


### Predictions

In [40]:
def sentiment_prediction(review):
    seq = tokenizer.texts_to_sequences([review]) 
    padded_seq = pad_sequences(seq, maxlen=300)
    
    prediction = model.predict(padded_seq)
    #prediction is a scole between 0 and 1, with 0 being a negative review and 1 being a positive review
    
    if prediction[0][0] > 0.6: 
        sentiment = "This is a positive review."
    elif prediction[0][0] < 0.4:
        sentiment = "This is a negative review."
    else:
        sentiment = "This review is not strongly negative or positive."
    
    return sentiment

In [41]:
sentiment_prediction("It was not my favourite movie")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step


'This review is not strongly negative or positive.'

In [42]:
sentiment_prediction("I hated every minute of this movie")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


'This is a negative review.'

In [43]:
sentiment_prediction("The costuming in this movie was great, and the art direction was some of my favourite")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


'This is a positive review.'

In [44]:
sentiment_prediction("This movie is fantastic")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


'This is a positive review.'